In [1]:
import zipfile
import os

# Path to the downloaded ZIP file
zip_path = "archive.zip"  # Update if path is different
extract_to = "cifake"  # Folder where files will be extracted

# Unzip the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("✅ Dataset unzipped to:", extract_to)


✅ Dataset unzipped to: cifake


In [7]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.7/252.7 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 12.0 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 12.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.4/670.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 10.3 MB/s eta 0:00:00a 0:00:01


In [8]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam

# Path to manually unzipped folder
base_dir = 'cifake'  # Update if you renamed it

# Parameters
img_height, img_width = 32, 32
batch_size = 32

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

Found 96000 images belonging to 2 classes.
Found 24000 images belonging to 2 classes.


In [11]:
def create_custom_cnn():
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, 3)),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

cnn_model = create_custom_cnn()
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

cnn_model.fit(train_generator, epochs=10, validation_data=val_generator)

Epoch 1/10


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3000/3000 ━━━━━━━━━━━━━━━━━━━━ 43s 14ms/step - accuracy: 0.8342 - loss: 0.4540 - val_accuracy: 0.8333 - val_loss: 0.4506
Epoch 2/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 45s 15ms/step - accuracy: 0.8352 - loss: 0.4487 - val_accuracy: 0.8333 - val_loss: 0.4543
Epoch 3/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 47s 16ms/step - accuracy: 0.8343 - loss: 0.4502 - val_accuracy: 0.8333 - val_loss: 0.4508
Epoch 4/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 48s 16ms/step - accuracy: 0.8315 - loss: 0.4542 - val_accuracy: 0.8333 - val_loss: 0.4505
Epoch 5/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 48s 16ms/step - accuracy: 0.8340 - loss: 0.4500 - val_accuracy: 0.8333 - val_loss: 0.4507
Epoch 6/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 49s 16ms/step - accuracy: 0.8318 - loss: 0.4537 - val_accuracy: 0.8333 - val_loss: 0.4517
Epoch 7/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 49s 16ms/step - accuracy: 0.8332 - loss: 0.4513 - val_accuracy: 0.8333 - val_loss: 0.4511
Epoch 8/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 53s 18ms/step - accuracy: 0.8351 - loss: 0.44

In [13]:
def create_resnet_model():
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3))
    base_model.trainable = False
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

resnet_model = create_resnet_model()
resnet_model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

resnet_model.fit(train_generator, epochs=10, validation_data=val_generator)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step
Epoch 1/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 220s 73ms/step - accuracy: 0.7951 - loss: 0.4954 - val_accuracy: 0.8333 - val_loss: 0.4520
Epoch 2/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 204s 68ms/step - accuracy: 0.8329 - loss: 0.4525 - val_accuracy: 0.8333 - val_loss: 0.4514
Epoch 3/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 205s 68ms/step - accuracy: 0.8334 - loss: 0.4514 - val_accuracy: 0.8333 - val_loss: 0.4511
Epoch 4/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 204s 68ms/step - accuracy: 0.8325 - loss: 0.4528 - val_accuracy: 0.8333 - val_loss: 0.4510
Epoch 5/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 212s 71ms/step - accuracy: 0.8332 - loss: 0.4519 - val_accuracy: 0.8333 - val_loss: 0.4522
Epoch 6/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 212s 71ms/step - accuracy: 0.8333 - loss: 0.4515 - val_accuracy: 0.8333 - val_loss: 0.4510
Epoch 7/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 203s 68ms/step - accuracy: 0.8325 - loss: 0.4529 - val_accuracy: 0.8333 - val_loss: 0.4519
Epoch 8/10
3000/30

In [27]:
cnn_model = create_custom_cnn()
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])